In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.keras.backend.clear_session()
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

# 模型构造
inputs = keras.Input(shape=(784,), name='mnist_input')
h1 = layers.Dense(64, activation='relu')(inputs)
h1 = layers.Dense(64, activation='relu')(h1)
outputs = layers.Dense(10, activation='softmax')(h1)
model = keras.Model(inputs, outputs)
# keras.utils.plot_model(model, 'net001.png', show_shapes=True)

model.compile(optimizer=keras.optimizers.RMSprop(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=[keras.metrics.SparseCategoricalAccuracy()])

# 载入数据
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') /255
x_test = x_test.reshape(10000, 784).astype('float32') /255

# 保证还是float 32？ 否则后面会出现：TypeError: Input 'y' of 'Sub' Op has type float32 that does not match type uint8 of argument 'x'.
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')



# 取验证数据
x_val = x_train[-10000:]
y_val = y_train[-10000:]

x_train = x_train[:-10000]
y_train = y_train[:-10000]

# 训练模型
history = model.fit(x_train, y_train, batch_size=64, epochs=3,
         validation_data=(x_val, y_val))
print('history:')
print(history.history)

result = model.evaluate(x_test, y_test, batch_size=128)
print('evaluate:')
print(result)
pred = model.predict(x_test[:2])
print('predict:')
print(pred)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 0.3383 - sparse_categorical_accuracy: 0.9044 - val_loss: 0.2192 - val_sparse_categorical_accuracy: 0.9359
Epoch 2/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1598 - sparse_categorical_accuracy: 0.9525 - val_loss: 0.1335 - val_sparse_categorical_accuracy: 0.9615
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1139 - sparse_categorical_accuracy: 0.9658 - val_loss: 0.1213 - val_sparse_categorical_accuracy: 0.9664
history:
{'loss': [0.33833765983581543, 0.15983255207538605, 0.11385840177536011], 'sparse_categorical_accuracy': [0.9043800234794617, 0.9525399804115295, 0.9657800197601318], 'val_loss': [0.21916982531547546, 0.13354036211967468, 0.12130691856145859], 'val_sparse_categorical_accuracy': [0.9358999729156494, 0.9614999890327454, 0.9664000272750854]}
79/79 [==============================] - 0s 1ms/

In [2]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [3]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name='digits')
    x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = layers.Dense(64, activation='relu', name='dense_2')(x)
    outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model
model = get_uncompiled_model()

def basic_loss_function(y_true, y_pred):
    return tf.math.reduce_mean(y_true - y_pred)
model.compile(optimizer=keras.optimizers.Adam(),
              loss=basic_loss_function)
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 4.3488
Epoch 2/3
782/782 [==============================] - 2s 2ms/step - loss: 4.3488
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 4.3488


In [4]:
    def __init__(self, pos_weight, weight, from_logits=False,
                reduction=keras.losses.Reduction.AUTO,
                name='weight_binary_crossentropy'):
        """
        pos_weight: 正类标签权重
        weight: 整体损失权重
        from_logits: 是否使用logits来计算loss，（或使用probability）
        reduction: reduction类型
        name: 名字
        """
        super(WeightBinaryCrossEntropy, self).__init__(reduction=reduction,
                                                      name=name)
        self.pos_weight = pos_weight
        self.weight = weight
        self.from_logits = from_logits
        
    def call(self, y_true, y_pred):
        if not self.from_logits:
            x_1 = y_true * self.pos_weight * -tf.math.log(y_pred + 1e-6)
            
            x_2 = (1-y_true) * -tf.math.log(1-y_pred + 1e-6)
            
            return tf.add(x_1, x_2) * self.weight
        
        return tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, self.pos_weight) *self.weight
        

model.compile(optimizer=keras.optimizers.Adam(),
             loss=WeightBinaryCrossEntropy(0.5, 2))
model.fit(x_train, y_train, batch_size=64, epochs=3)

NameError: ignored

In [5]:
# 下面是一个简单的示例，显示如何实现CatgoricalTruePositives指标，该指标计算正确分类为属于给定类的样本数量

class CatgoricalTruePostives(keras.metrics.Metric):
    def __init__(self, name='binary_true_postives', **kwargs):
        super(CatgoricalTruePostives, self).__init__(name=name, **kwargs)
        # 会更新的类变量
        self.true_postives = self.add_weight(name='tp', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        # 获取结果id
        y_pred = tf.argmax(y_pred)
        # 正确的结果
        y_true = tf.equal(tf.cast(y_pred, tf.int32), tf.cast(y_true, tf.int32))
        y_true = tf.cast(y_true, tf.float32)
        
        if sample_weight is not None:
            # 对正确结果加权重
            sample_weight = tf.cast(sample_weight, tf.float32)
            y_true = tf.multiply(sample_weight, y_true)
        # 修改正确样本总量
        return self.true_postives.assign_add(tf.reduce_sum(y_true))
    
    def result(self):
        # 返回相应tensor
        return tf.identity(self.true_postives)
    
    def reset_states(self):
        # 重置为0
        self.true_postives.assign(0.)
        

model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=[CatgoricalTruePostives()])

model.fit(x_train, y_train,
         batch_size=64, epochs=3)

Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 0.3426 - binary_true_postives: 7594.0000
Epoch 2/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1645 - binary_true_postives: 7753.0000
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1210 - binary_true_postives: 7666.0000
